# Fluxo Novo

Ordem das células

- Importando base de dados
- Definindo lojas que estarão no relatório
- Definindo dia do relatório
- Definindo emails que receberão o relatório
- Rodando o código para a criação do relatório
- Preparando o html
- Rodando Mock do disparador de email e disparando email

## Importando base de dados


In [49]:
# -> Rodar tudo para ter os dataframes corretos
from commom.database.queries.query_lojas import QUERY_LOJAS
from commom.database.queries.query_vendas_nf_novo import QUERY_VENDAS_NF_NOVO
from commom.database.queries.query_metas import QUERY_METAS
from commom.database.queries.query_parcelas import QUERY_PARCELAS
from commom.database.queries.query_vendas_nf import QUERY_VENDAS_NF
from commom.database.queries.query_vendedores import QUERY_VENDEDORES
from commom.database.queries.query_loja_vendedores import QUERY_LOJA_VENDEDORES
from commom.kpi_data.kpi_data_manager import DataHandler, KpiDataManager
from typing import Tuple
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from innovation_messenger import Messenger
import os
from innovation_messenger import IEmailProperties
import missingno as msno

manager = KpiDataManager()
handler = DataHandler()

# Service
## -> inicio
###     -> fetch data from local pickle
local_df_fato_vendas = handler.read_from_local_pickle('df_fato_vendas.pkl')
local_df_lojas = handler.read_from_local_pickle('df_lojas.pkl')
local_df_vendedores = handler.read_from_local_pickle('df_vendedores.pkl')
local_df_vendas = handler.read_from_local_pickle('df_vendas.pkl')
local_df_parcelas = handler.read_from_local_pickle('df_parcelas.pkl')
local_df_metas = handler.read_from_local_pickle('df_metas.pkl')
local_df_vendedor_loja = handler.read_from_local_pickle('df_vendedor_loja.pkl')

###     -> fetch data from bq
# local_df_fato_vendas = handler.read_from_bigquery(QUERY_VENDAS_NF_NOVO)
# local_df_lojas = handler.read_from_bigquery(QUERY_LOJAS)
# local_df_vendedores = handler.read_from_bigquery(QUERY_VENDEDORES)
# local_df_vendas = handler.read_from_bigquery(QUERY_VENDAS_NF)
# local_df_parcelas = handler.read_from_bigquery(QUERY_PARCELAS)
# local_df_metas = handler.read_from_bigquery(QUERY_METAS).rename(
#     columns={"cpf_vendedor": "cpf_vendedor_inteiro", "loja": "distributorId"}
# )
# local_df_vendedor_loja = handler.read_from_bigquery(QUERY_LOJA_VENDEDORES)

## Definindo lojas que estarão no relatório

In [50]:
storesList = ['10H1', '1077', '10I2', '1045', '10J4', '2569', '10C3', '2584', '2590', '0595', '10G1', '2582']
print('lojas', storesList)

lojas ['10H1', '1077', '10I2', '1045', '10J4', '2569', '10C3', '2584', '2590', '0595', '10G1', '2582']


## Definindo data do relatório

In [51]:
# reportDate = datetime.datetime.now()
reportDate = datetime.datetime(2025,1,18)
print('dia escolhido ', reportDate)

dia escolhido  2025-01-18 00:00:00


## Rodando código para criação do relatório

In [52]:
### - Funções base [cria df_vendas_pdv, cria mascaras(vendas_loja | vendas_vendedor | meta), faz o merge(vendas-CPFvendedor e CPFvendedor-nome) ]

def _abreviate_vendedor_name(full_name: str) -> str:
    full_name = full_name.replace("   ", " ")
    full_name = full_name.replace("  ", " ")

    if len(full_name.title().split(" ")) > 1:
        splited_name = full_name.title().split(" ")
        nome_completo = splited_name[0]
        splited_name.pop(0)

        for name in splited_name:
            nome_completo += " " + name[0] + "."

        return nome_completo

    return full_name

# ----------------------------------------
# ----------------------------------------

def create_vendas_pdv():
    df_vendas_temp = local_df_fato_vendas.copy()

    PDV_TYPE_LIST = ['Propria']

    mask_pdv = df_vendas_temp["type"].isin(PDV_TYPE_LIST)
    maksNull = df_vendas_temp['loja_esforco'].isnull()
    df_vendas_pdv = df_vendas_temp[mask_pdv & ~maksNull]
    df_vendas_pdv = df_vendas_pdv.copy()
    df_vendas_pdv['createdAt'] = pd.to_datetime(df_vendas_pdv['createdAt'])
    
    renameDict = {
        'displayCode': 'displaycode',
        'vendedor_esforco': 'cpf_vendedor_inteiro',
        'esforco_data': 'invoice_date'
    }

    df_vendas_pdv = df_vendas_pdv.reset_index(drop=True)
    
    df_vendas_pdv['year'] = df_vendas_pdv['createdAt'].dt.year
    df_vendas_pdv['month'] = df_vendas_pdv['createdAt'].dt.month
    df_vendas_pdv['day'] = df_vendas_pdv['createdAt'].dt.day
    
    df_vendas_pdv.rename(columns=renameDict, inplace=True)
    return df_vendas_pdv

# ----------------------------------------
# ----------------------------------------

def merge_kpiVendedor_nome(df_vendedores:pd.DataFrame, vendedor_loja:pd.DataFrame, df_kpiVendedor_filtro: pd.DataFrame, id_loja: str) -> pd.DataFrame:
    df_nome_vendedor = df_vendedores[["name", "cpf"]].copy()
    df_nome_vendedor.columns = ["vendedor", "cpf_vendedor_inteiro"]
    df_nome_vendedor.loc[:, "vendedor"] = df_nome_vendedor["vendedor"].apply(_abreviate_vendedor_name)
    
    maskvendedorLoja = vendedor_loja['distributorId'] == id_loja
    vendedor_loja = vendedor_loja[maskvendedorLoja].copy()
    listCpf = vendedor_loja['employeeCpf'].tolist()
    
    maskCpf = df_kpiVendedor_filtro['cpf_vendedor_inteiro'].isin(listCpf)
    df_kpiVendedor_filtro = df_kpiVendedor_filtro[maskCpf]
    
    df_kpiVendedor_filtro = df_nome_vendedor.merge(df_kpiVendedor_filtro, on="cpf_vendedor_inteiro").drop(
        columns="cpf_vendedor_inteiro"
    )
    
    return df_kpiVendedor_filtro

def build_vendedor():
    df_nome_vendedor = local_df_vendedores[["name", "cpf"]]
    df_nome_vendedor.columns = ["vendedor", "cpf_vendedor_inteiro"]
    df_nome_vendedor.loc[:, "vendedor"] = df_nome_vendedor["vendedor"].apply(_abreviate_vendedor_name)
    return df_nome_vendedor

# ----------------------------------------
# ----------------------------------------

def create_masks_meta_vendedor(periodo:str, df_vendas_:pd.DataFrame , df_metas:pd.DataFrame, date: datetime.datetime):
    mask_meta = df_metas["tipo_meta"] == "Meta Vendedor PDV"
    mask_vendas = pd.Series(np.ones(df_vendas_.shape[0]).astype(bool))
    
    if periodo == "Dia":
        mask_yesterday_vendas = df_vendas_["invoice_date"] == date
        mask_vendas = mask_vendas & mask_yesterday_vendas
        
        mask_metas_yesterday = df_metas["date"] == date
        mask_meta = mask_meta & mask_metas_yesterday

    return mask_meta, mask_vendas

# ----------------------------------------
# ----------------------------------------

def create_masks_meta_loja(periodo:str,  df_vendas_:pd.DataFrame , df_metas:pd.DataFrame, date: datetime.datetime):
    mask_meta = df_metas["tipo_meta"] == "Meta Loja PDV"
    mask_vendas = pd.Series(np.ones(df_vendas_.shape[0]).astype(bool))
    
    if periodo == "Dia":
        mask_yesterday_vendas = df_vendas_["invoice_date"] == date
        mask_vendas = mask_vendas & mask_yesterday_vendas
        
        mask_metas_yesterday = df_metas["date"] == date
        mask_meta = mask_meta & mask_metas_yesterday

    return mask_meta, mask_vendas


In [53]:
### - Função para criar o df_kpi_vendedor
def create_kpi_vendedor_df(kpi_period_:str, df_metas: pd.DataFrame, date: datetime.datetime, df_vendas_pdv_: pd.DataFrame) -> pd.DataFrame:
    maskmeta, maskvenda = create_masks_meta_vendedor(periodo=kpi_period_, df_vendas_=df_vendas_pdv_, df_metas=df_metas, date=date)
    AGG_DICT = { 
        "discount": "sum", 
        "netValue": "sum", 
        "cost": "sum" 
    }

    GROUP_BY_COLUMNS_DICT = {
        "Ano": ["loja_esforco", "year","cpf_vendedor_inteiro"],
        "Mês": ["loja_esforco", "year", "month", "cpf_vendedor_inteiro"],
        "Dia": ["loja_esforco", "year", "month", "day", "cpf_vendedor_inteiro"],
    }

    group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period_]

    df_kpis: pd.DataFrame = df_vendas_pdv_[maskvenda].groupby(group_by_columns).agg(AGG_DICT).reset_index()

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskvenda].groupby(group_by_columns)["fiscalsParentOrder"].nunique().reset_index().rename(columns={"fiscalsParentOrder": "tkts"}),
        how="left",
    )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskvenda].groupby(group_by_columns)["amount"].sum().reset_index(), 
        how="left"
    )

    metas_df = df_metas[maskmeta].copy()
    metas_df.rename(columns={"distributorId": "loja_esforco"}, inplace=True)
    
    # dfmodificado = df_kpis.merge(
    #     metas_df,
    #     how="left"
    # )
    
    df_kpis = df_kpis.merge(
        metas_df.groupby(group_by_columns)["meta"].sum().reset_index(), 
        how="left"
    )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_.groupby(group_by_columns)["invoice_date"]
        .nunique()
        .reset_index()
        .rename(columns={"invoice_date": "days_of_work"}),
        how="left",
    )

    if kpi_period_ == "Dia":
        df_kpis["Dia"] = reportDate.day

    if kpi_period_ == "Mês":
        df_kpis["tkts_per_day"] = df_kpis["tkts"] / df_kpis["days_of_work"].round(2)
    else:
        df_kpis["tkts_per_day"] = df_kpis["tkts"]

    df_kpis["type"] = kpi_period_

    df_kpis["mkp"] = (df_kpis["netValue"] / df_kpis["cost"]).round(2)

    df_kpis["tkt_medio"] = df_kpis["netValue"] / df_kpis["tkts"].round(2)
    df_kpis["meta_percentual"] = (df_kpis["netValue"] / df_kpis["meta"] * 100).round(2)
    df_kpis["PA"] = df_kpis["amount"] / df_kpis["tkts"].round(2)
    df_kpis["discount_percentual"] = ((df_kpis["discount"] / df_kpis["netValue"]) * 100).round(2)
    
    df_kpis.replace({'mkp': [np.inf, -np.inf]}, 0, inplace=True)

    return df_kpis

In [54]:
### - Função para criar o df_kpi_loja
def create_kpi_loja_df(kpi_period_:str, df_metas: pd.DataFrame, date: datetime.datetime, df_vendas_pdv_: pd.DataFrame) -> pd.DataFrame:
    maskMeta, maskVendas = create_masks_meta_loja(periodo=kpi_period_, df_vendas_= df_vendas_pdv_, df_metas=df_metas, date=date)
    AGG_DICT = {"discount": "sum", "netValue": "sum", "cost": "sum"}

    GROUP_BY_COLUMNS_DICT = {
        "Ano": ["loja_esforco", "year"],
        "Mês": ["loja_esforco", "year", "month"],
        "Dia": ["loja_esforco", "year", "month", 'day'],
    }

    group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period_]

    df_kpis: pd.DataFrame = df_vendas_pdv_[maskVendas].groupby(group_by_columns).agg(AGG_DICT).reset_index()

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskVendas].groupby(group_by_columns)["fiscalsParentOrder"].nunique().reset_index().rename(columns={"fiscalsParentOrder": "tkts"}),
        how="left",
        )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskVendas].groupby(group_by_columns)["amount"].sum().reset_index(), 
        how="left"
    )

    metas_df = df_metas[maskMeta].copy()
    metas_df.rename(columns={"distributorId": "loja_esforco"}, inplace=True)
    
    dfmodificado = df_kpis.merge(
        metas_df,
        how="left"
    )
    
    df_kpis = df_kpis.merge(
        dfmodificado.groupby(group_by_columns)["meta"].sum().reset_index(), 
        how="left"
    )
    
    if kpi_period_ == "Dia":
        df_kpis["Dia"] = date.day

    if kpi_period_ == "Dia":
        df_kpis["tkts_per_day"] = df_kpis["tkts"]
    if kpi_period_ == "Mês":
        df_kpis["tkts_per_day"] = df_kpis["tkts"] / date.day
    if kpi_period_ == "Ano":
        df_kpis["tkts_per_day"] = df_kpis["tkts"] / date.timetuple().tm_yday 
        
    df_kpis["type"] = kpi_period_
    df_kpis["mkp"] = (df_kpis["netValue"] / df_kpis["cost"]).round(2)
    df_kpis["tkt_medio"] = (df_kpis["netValue"] / df_kpis["tkts"]).round(2)
    df_kpis["meta_percentual"] = (df_kpis["netValue"] / df_kpis["meta"] * 100).round(2)
    df_kpis["PA"] = (df_kpis["amount"] / df_kpis["tkts"]).round(2)
    df_kpis["discount_percentual"] = ((df_kpis["discount"] / df_kpis["netValue"]) * 100).round(2)

    df_kpis.replace({'mkp': [np.inf, -np.inf]}, 0, inplace=True)
    
    return df_kpis

In [55]:
### - Função para formatar o kpi_loja
def fix_idx(value: str)->int:
    if value == "Dia":
        return 0
    elif value == "Mês":
        return 1
    elif value == "Ano":
        return 2
    
def format_kpi_loja(id_loja: str, df_kpi:pd.DataFrame, date: datetime.datetime):
    mask = df_kpi["loja_esforco"] == id_loja
    df_store = df_kpi[mask].fillna(0).copy()
    mask = df_store['year'] == date.year
    df_store = df_store[mask]
    
    df_store_current_month = df_store.loc[df_store.groupby(["type"])["month"].idxmax()].drop_duplicates()
    
    rename_dict = {
        "netValue": "Venda Loja (PDV)",
        "meta": "Meta Loja (PDV)",
        "meta_percentual": "Meta % (PDV)",
        "mkp": "Markup",
        "tkts_per_day": "Tickets/Dia",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        "discount_percentual": "Desconto Médio %",
        "type": "Período",
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Venda Loja (PDV)": format_money,
        "Meta Loja (PDV)": format_money,
        "Meta % (PDV)": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets/Dia": "{:,.1f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        "Desconto Médio %": format_percentage,
    }

    df_store_current_month['idx_helper'] = df_store_current_month['type'].apply(fix_idx)
    df_store_current_month.sort_values(by='idx_helper', ascending=True, inplace=True)
    df_store_current_month.drop(columns='idx_helper')
    
    df_store_formated = df_store_current_month[rename_dict.keys()].round(2).rename(columns=rename_dict)
    
    for key, format in format_dict.items():
        df_store_formated[key] = df_store_formated[key].map(format.format)

        if format == format_money:
            df_store_formated[key] = df_store_formated[key].astype(str).str.replace(",", ".")

    try:
        df_store_formated = df_store_formated.set_index("Período").reset_index().T
    except KeyError:
        df_store_formated = pd.DataFrame()
    
    return df_store_formated

In [56]:
### - Função para formatar o kpi_vendedor (dia | mes)
def format_kpi_vendedor(id_loja:str, kpi_df_vendedor:pd.DataFrame, vendedor_loja_df:pd.DataFrame, df_vendedores: pd.DataFrame,periodo: str, date: datetime.datetime):
    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"
    
    mask = kpi_df_vendedor["loja_esforco"] == id_loja
    df_vendedor = kpi_df_vendedor[mask].reset_index(drop=True)
    df_vendedor["Mês"] = df_vendedor["month"].fillna(0)

    mask_mes = df_vendedor["Mês"] == date.month
    mask_ano = df_vendedor["year"] == date.year
    
    if periodo == "Dia":
        mask_type = df_vendedor["type"].isin(["Dia"])
        mask_dia = df_vendedor['Dia'] == date.day
        mask_completa = mask_type & mask_dia & mask_mes & mask_ano
    else:
        mask_type = df_vendedor["type"].isin(["Mês"])
        mask_completa = mask_type & mask_mes & mask_ano
        
    df_vendedor_filtro: pd.DataFrame = df_vendedor[mask_completa].reset_index(drop=True)
    
    df_vendedor_filtro["normalized_net_value"] = (
        df_vendedor_filtro["netValue"] * 100 / round(df_vendedor_filtro["netValue"].max(), 2)
    )
    
    df_vendedor_filtro["net_value_share"] = (
        df_vendedor_filtro["netValue"] * 100 / round(df_vendedor_filtro["netValue"].max(), 2)
    )
    
    df_vendedor_filtro = merge_kpiVendedor_nome(
        vendedor_loja=vendedor_loja_df, 
        df_vendedores=df_vendedores, 
        df_kpiVendedor_filtro=df_vendedor_filtro, 
        id_loja=id_loja
    )
    
    if periodo == "Dia":
        rename_dict = {
            "vendedor": "Vendedor",
            "netValue": "Acumulado no Dia",
            "normalized_net_value": "Homogeneidade",
            "meta": "Meta do Dia",
            "meta_percentual": "Meta %",
            "mkp": "Markup",
            "tkts_per_day": "Tickets",
            "PA": "Peças por Atendimento",
            "tkt_medio": "Ticket Médio",
            "discount_percentual": "Desconto Médio %",
        }
        
        format_dict = {
            "Acumulado no Dia": format_money,
            "Homogeneidade": format_percentage,
            "Meta do Dia": format_money,
            "Meta %": format_percentage,
            "Markup": "{:,.2f}",
            "Tickets": "{:,.0f}",
            "Peças por Atendimento": "{:,.1f}",
            "Ticket Médio": format_money,
            "Desconto Médio %": format_percentage,
        }
        
        df_vendedor_formatado = (
            df_vendedor_filtro[rename_dict.keys()]
            .sort_values("normalized_net_value", ascending=False)
            .rename(columns=rename_dict)
        )
        
    else:
        rename_dict = {
            "vendedor": "Vendedor",
            "days_of_work": "Dias Trabalhados",
            "netValue": "Acumulado no Mês",
            "normalized_net_value": "Homogeneidade",
            "meta": "Meta do Mês",
            "meta_percentual": "Meta %",
            "mkp": "Markup",
            "tkts_per_day": "Tickets/Dia",
            "PA": "Peças por Atendimento",
            "tkt_medio": "Ticket Médio",
            "discount_percentual": "Desconto Médio %",
        }
        
        format_dict = {
            "Acumulado no Mês": format_money,
            "Homogeneidade": format_percentage,
            "Meta do Mês": format_money,
            "Meta %": format_percentage,
            "Markup": "{:,.2f}",
            "Tickets/Dia": "{:,.1f}",
            "Peças por Atendimento": "{:,.1f}",
            "Ticket Médio": format_money,
            "Desconto Médio %": format_percentage,
        }

        df_vendedor_formatado = (
            df_vendedor_filtro[rename_dict.keys()]
            .sort_values("normalized_net_value", ascending=False)
            .round(2)
            .rename(columns=rename_dict)
        )

    for key, format in format_dict.items():
        df_vendedor_formatado[key] = df_vendedor_formatado[key].map(format.format)
        if format == format_money:
            df_vendedor_formatado[key] = df_vendedor_formatado[key].astype(str).str.replace(",", ".")

    df_vendedor_formatado = df_vendedor_formatado.reset_index(drop=True).T.reset_index().T

    return df_vendedor_formatado.set_index(0)

In [57]:
### - Funções para criar uma lista de kpi por periodo de tempo (Dia | Mês | Ano)
def create_kpi_loja_period_list():
    df_vendas_pdv = create_vendas_pdv()
    kpis_loja_dict_ = {}
    
    for kpi_period_ in ["Dia", "Mês", "Ano"]:
        kpis_loja_dict_[f"kpis_loja_{kpi_period_.lower()}"] = create_kpi_loja_df(kpi_period_=kpi_period_, df_vendas_pdv_=df_vendas_pdv, df_metas=local_df_metas, date=reportDate)
    
    df_kpis_loja_list = pd.concat(list(kpis_loja_dict_.values()))
    
    return df_kpis_loja_list

def create_kpi_vendedor_period_list():
    df_vendas_pdv = create_vendas_pdv()
    kpis_vendedor_dict_ = {}
        
    for kpi_period_ in ["Dia", "Mês", "Ano"]:
        kpis_vendedor_dict_[f"kpis_vendedor_{kpi_period_.lower()}"] = create_kpi_vendedor_df(kpi_period_=kpi_period_, df_vendas_pdv_=df_vendas_pdv, df_metas=local_df_metas, date=reportDate)

    df_kpis_vendedor_list = pd.concat(list(kpis_vendedor_dict_.values()))
    
    return df_kpis_vendedor_list


In [58]:
### - Função para corrigir vendas vazias para o dia
def fix_kpi_LojaList(id_loja:str, date: datetime.date, df_loja_list: pd.DataFrame, df_metas: pd.DataFrame):
    df = df_loja_list.copy()
    df = df.fillna(0).copy()

    tipoMetaMask = df_metas['tipo_meta'] == 'Meta Loja PDV'
    lojaMetaMask = df_metas['distributorId'] == id_loja
    dataMetaMask = df_metas['date'] == date
    df_metas = df_metas[tipoMetaMask].copy()
    df_metas = df_metas[lojaMetaMask].copy()
    df_metas = df_metas[dataMetaMask].copy()

    lojaMask = df['loja_esforco']==id_loja

    typeDayMask = df['type'] == 'Dia'
    typeMonthMask = df['type'] == 'Mês'
    typeYearMask = df['type'] == 'Ano'
    yearMask = df['year'] == date.year
    monthMask = df['month'] == date.month
    dayMask = df['day'] == date.day

    typeYear = lojaMask & typeYearMask & yearMask
    typeMonth = lojaMask & yearMask & typeMonthMask & monthMask
    typeDay = lojaMask & yearMask & typeDayMask & monthMask & dayMask

    dfAno = df[typeYear]
    dfMes = df[typeMonth]

    dfDia = df[typeDay]
    if dfDia.empty:
        dfDia['meta'] = df_metas['meta']
        
        dfDia.fillna(0, inplace=True)
        dfDia.loc[:,'loja_esforco'] = id_loja
        dfDia.loc[:,'year'] = date.year
        dfDia.loc[:,'month'] = date.month
        dfDia.loc[:,'day'] = date.day
        dfDia.loc[:,'type'] = 'Dia'

    return pd.concat([dfAno, dfMes, dfDia]).reset_index()


In [59]:
### - Criação dos kpis por periodo 
df_kpis_loja_list = create_kpi_loja_period_list()
df_kpis_vendedor_list = create_kpi_vendedor_period_list()

In [60]:
### - Debug
### $ $ - Cria uma lista de lojas aleatórias entre as que estão disponíveis no df
# import random
# anoMascara = df_kpis_loja_list['year'] == reportDate.year
# mesMascara = df_kpis_loja_list['month'] == reportDate.month
# df_storesForDate = df_kpis_loja_list[anoMascara & mesMascara]
# listOfCurrentStores = df_storesForDate['loja_esforco'].unique()

# storesList = random.choices(listOfCurrentStores, k=7)
# print(storesList)

In [61]:
### - Formatando todos os kpis
kpis_loja_formated_dict_ = {}
kpis_vendedor_dia_formated_dict_ = {}
kpis_vendedor_mes_formated_dict_ = {}

for loja in storesList:
    kpis_loja_formated_dict_[f"{loja}"] = format_kpi_loja(
        id_loja=loja, 
        df_kpi=fix_kpi_LojaList(id_loja=loja, date= reportDate, df_loja_list= df_kpis_loja_list, df_metas=local_df_metas),
        date=reportDate
    )
    
    kpis_vendedor_dia_formated_dict_[f"{loja}"] = format_kpi_vendedor(
        id_loja=loja,
        kpi_df_vendedor=df_kpis_vendedor_list,
        vendedor_loja_df=local_df_vendedor_loja,
        df_vendedores=local_df_vendedores,
        periodo='Dia',
        date=reportDate
    )
    
    kpis_vendedor_mes_formated_dict_[f"{loja}"] = format_kpi_vendedor(
        id_loja=loja,
        kpi_df_vendedor=df_kpis_vendedor_list,
        vendedor_loja_df=local_df_vendedor_loja,
        df_vendedores=local_df_vendedores,
        periodo='Mês',
        date=reportDate
    )


/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_3330/2132309198.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_metas = df_metas[lojaMetaMask].copy()
/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_3330/2132309198.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_metas = df_metas[dataMetaMask].copy()
/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_3330/2132309198.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_metas = df_metas[lojaMetaMask].copy()
/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_3330/2132309198.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_metas = df_metas[dataMetaMask].copy()
/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_3330/2132309198.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_metas = df_metas[lojaMetaMask].co

In [62]:
### - Função de criação do html para uma conjunto <loja | vendedor>
def create_html_text_for_df(id_loja:str, date: datetime.datetime, df_loja: pd.DataFrame, df_vendedor_dia: pd.DataFrame, df_vendedor_mes: pd.DataFrame ) -> str :
    relatorioDiaStr = date.strftime("%d-%m-%Y")
    emailBody = ""

    htmlValue = """
    <html>
    <head></head>
    <body>
        <h1>Relatorio loja - {1}</h1>
        <h2>Data: {2}</h2>
        {0}
    </body>
    <br/><br/>
    </html>
    """.format(df_loja.to_html(), id_loja, date.strftime('%d-%m-%Y'))
    emailBody += htmlValue

    htmlValue = """
    <html>
    <head></head>
    <body>
    <h1>Relatorio vendedor - Visão diária</h1>
    <p>Loja:{1} | Data: {2}</p>
        {0}
    </body>
    <br/><br/>
    </html>
    """.format(df_vendedor_dia.to_html(),id_loja, date.strftime('%d-%m-%Y'))
    emailBody += htmlValue

    htmlValue = """
    <html>
    <head></head>
    <body>
    <h1>Relatorio vendedor - Visão mensal</h1>
    <p>Loja:{1} | Data: {2}</p>
        {0}
    </body>
    <br/><br/>
    <p>-------------------------------------------------------------</p>
    <p>-------------------------------------------------------------</p>
    </html>
    """.format(df_vendedor_mes.to_html(),id_loja, date.strftime('%m-%Y'))
    emailBody += htmlValue
    
    return emailBody


In [63]:
### - Definindo conteúdo html e recipients
fullHtml = ""
for store in storesList:
    loja = kpis_loja_formated_dict_[f'{store}']
    dia = kpis_vendedor_dia_formated_dict_[f'{store}']
    mes = kpis_vendedor_mes_formated_dict_[f'{store}']
    fullHtml += create_html_text_for_df(
        id_loja=store, 
        date=reportDate, 
        df_loja=loja, 
        df_vendedor_dia=dia, 
        df_vendedor_mes=mes 
    )

relatorioDiaStr = reportDate.strftime("%d-%m-%Y")
emailRecipients = ['joao.garcia@ammovarejo.com.br']
print('recipientes: ', emailRecipients)    

recipientes:  ['joao.garcia@ammovarejo.com.br']


In [64]:
import os
import smtplib
from dataclasses import dataclass
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from typing import Any, Callable, Dict, List, Optional, Union
from commom.logger import logger
from commom.base_classes.base_sender import BaseMessenger
from innovation_messenger.config import config

@dataclass
class IEmailProperties:
    subject: str
    body: str
    recipient: Union[str, List[str]]
    file_name: Optional[str] = None


def send_email_html(email_properties: IEmailProperties) -> bool:
    message = MIMEMultipart()
    message["Subject"] = email_properties.subject
    message["From"] = 'inovacao@ammovarejo.com.br'
    messageTo: str

    if isinstance(email_properties.recipient, List):
        messageTo = ", ".join(email_properties.recipient)
    else:
        messageTo = email_properties.recipient

    message["To"] = messageTo
    html_part = MIMEText(email_properties.body, 'html')
    message.attach(html_part)
    if isinstance(email_properties.recipient, List):
        messageTo = ", ".join(email_properties.recipient)
    else:
        messageTo = email_properties.recipient
        
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        try:
            server.login(user='inovacao@ammovarejo.com.br', password='inovaAM!05')
            server.sendmail('inovacao@ammovarejo.com.br', email_properties.recipient, message.as_string())
            return True
        except Exception as e:
            logger.error(e)
            return False


email_properties = IEmailProperties(
    subject=f'Teste - Base nova - Dia {relatorioDiaStr}',
    recipient=emailRecipients,
    body=fullHtml,
)

send_email_html(
  email_properties=email_properties,
)


True